# Installing Packages


In [ ]:
!pip install xgboost
!pip install fbprophet
!pip install keras
!pip install tensorflow

# Load the Libraries

In [ ]:
import pandas as pd
import acndata_utils
import Optimization_utils
import pandas as pd
import StationData_utils
import StationForecast_utils

# Load the Input Data

In [ ]:
dfs = []
for file_path in [
    "D:\\Individual Project\\Data\\acndata_sessions_Caltech.json",
    "D:\\Individual Project\\Data\\acndata_sessions_JPL.json",
    "D:\\Individual Project\\Data\\acndata_sessions_Office_1.json"
]:
    df = acndata_utils.getdf(file_path)
    if df is not None:
        dfs.append(df)
    else:
        print(f"Warning: {file_path} could not be processed and will be skipped.")

# Concat and sort all dataframes to get whole new data

In [ ]:
if dfs:
    df = pd.concat(dfs).sort_values(by="_id").reset_index(drop=True)
    df = df.drop(columns=['_id'])
    # Add new '_id' column as index_number
    df.reset_index(inplace=True)
    df.rename(columns={'index': '_id'}, inplace=True)
    df = acndata_utils.clean_acndata(df)
    print(df.shape)
else:
    print("No valid data available to process.")

Save the Data. This is our user data.

In [ ]:
df.to_csv("D:\\Individual Project\\Data\\acndata.csv", index=False)

# Converting Data to Station Data



In [ ]:
df = StationData_utils.analyze_hourly_sessions(df)

In [ ]:
df.head()

convert to csv to store the data.

In [ ]:
df.to_csv("D:\\Individual Project\\Data\\stationdata.csv",index=False)

Plotting The Data

In [ ]:
StationData_utils.create_interactive_plot(df)

# Aggregating the data

In [ ]:
df = df.groupby('date').agg({
    'Average_Power': 'mean',
    'chargingUsers': 'sum',
    'standbyUsers': 'sum'
}).reset_index()
df.set_index('date', inplace=True)

# Ensure target column is numeric

In [ ]:
columns_to_check = ['Average_Power', 'chargingUsers', 'standbyUsers']
existing_columns = [col for col in columns_to_check if col in df.columns]

df[existing_columns] = df[existing_columns].apply(pd.to_numeric, errors='coerce')
df = df.dropna()

# Forecasting and plotting charts using ARIMA, SARIMAX, Prophet, Random Forest, Decision Tree, XGBoost and LSTM

In [ ]:
df_new = df.copy()
for column in ['Average_Power', 'chargingUsers', 'standbyUsers']:
    train, test = StationForecast_utils.Train_Test(df_new, column)
    print(f"Running models for {column}...")

    arima_preds = StationForecast_utils.arima_forecast(train, test)
    sarimax_preds = StationForecast_utils.sarimax_forecast(train, test)
    prophet_preds = StationForecast_utils.prophet_forecast(df_new,column)
    tree_preds_rf = StationForecast_utils.tree_based_forecast(train, test, "RandomForest")
    tree_preds_dt = StationForecast_utils.tree_based_forecast(train, test, "DecisionTree")
    tree_preds_xgb = StationForecast_utils.tree_based_forecast(train, test, "XGBoost")
    lstm_preds = StationForecast_utils.lstm_forecast(train, test)

    StationForecast_utils.plot_last_day_forecast(test, arima_preds, "ARIMA")
    StationForecast_utils.plot_last_day_forecast(test, sarimax_preds, "SARIMAX")
    StationForecast_utils.plot_last_day_forecast(test, prophet_preds, "Prophet")
    StationForecast_utils.plot_last_day_forecast(test, tree_preds_rf, "RandomForest")
    StationForecast_utils.plot_last_day_forecast(test, tree_preds_dt, "DecisionTree")
    StationForecast_utils.plot_last_day_forecast(test, tree_preds_xgb, "XGBoost")
    StationForecast_utils.plot_last_day_forecast(test, lstm_preds, "LSTM")

# What is the seasonal charging behavior of EV charging vehicles?

In [ ]:
data = df.copy()
preprocessed_data = StationData_utils.preprocess_data(data)
monthly_behavior, weekday_behavior, daily_behavior, yearly_behavior = StationData_utils.aggregate_behavior(preprocessed_data)

# Plot for 'chargingUsers'
StationData_utils.plot_behavior(monthly_behavior, weekday_behavior, daily_behavior, yearly_behavior, 'chargingUsers')

# Plot for 'standbyUsers'
StationData_utils.plot_behavior(monthly_behavior, weekday_behavior, daily_behavior, yearly_behavior, 'standbyUsers')



# How is daily, weekly, monthly and yearly Load Demand of EV charging?

In [ ]:
# Plot for 'Energy_Consumed'
StationData_utils.plot_behavior(monthly_behavior, weekday_behavior, daily_behavior, yearly_behavior, 'Energy_Consumed')

# What is the average duration of charging sessions, and how does it vary across different stations?

In [ ]:
df = pd.read_csv("D:\\Individual Project\\Data\\acndata.csv")


avg_time, station_avg_time = acndata_utils.calculate_charging_time_statistics(df)
print("Average charging time across all sessions:", avg_time)
print("\nAverage charging time per station:")
print(station_avg_time)

# Optimization of user, station Data

In [ ]:
acndata = pd.read_csv("D:\\Individual Project\\Data\\acndata.csv")
stationdata = pd.read_csv("D:\\Individual Project\\Data\\stationdata.csv")
electricity_pricing = pd.read_csv("D:\\Individual Project\\Data\\Historical_Electricity_Pricing_2019_2021.csv")


# Preprocess and compute necessary fields

In [ ]:
 acndata, stationdata = Optimization_utils.preprocess_data(acndata, stationdata)
stationdata = Optimization_utils.calculate_power_new(stationdata, acndata)
acndata, stationdata = Optimization_utils.calculate_session_energy(acndata, stationdata)
acndata, stationdata = Optimization_utils.link_pricing_data(acndata, stationdata, electricity_pricing)


# Perform joint optimization

In [ ]:
acndata, stationdata = Optimization_utils.fast_optimization(acndata, stationdata)

# Save results

In [ ]:
acndata.to_csv('optimized_acndata.csv', index=False)
stationdata.to_csv('optimized_stationdata.csv', index=False)

# What percentage of connected time is actually spent charging, and how does this vary across stations?

In [ ]:
Optimization_utils.generate_charging_time_chart(acndata)

# How are the tariffs across different period of time?

In [ ]:
Optimization_utils.plot_tariffs_over_time(electricity_pricing)

# What is the potential impact on grid load if we shift a percentage of charging sessions from peak to off-peak hours, and how would this affect overall charging costs?

Preprocess data

In [ ]:
stationdata['session_start'] = pd.to_datetime(stationdata['session_start'])
    stationdata['session_end'] = pd.to_datetime(stationdata['session_end'])
    stationdata['Year'] = stationdata['session_start'].dt.year
    stationdata['Season'] = stationdata['session_start'].dt.month.map({
        1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring',
        5: 'Spring', 6: 'Summer', 7: 'Summer', 8: 'Summer',
        9: 'Fall', 10: 'Fall', 11: 'Fall', 12: 'Winter'
    })

Original load and cost

In [ ]:
original_load, original_cost = Optimization_utils.calculate_grid_load_and_cost(stationdata, electricity_pricing)


Shift charging sessions

In [ ]:
shifted_stationdata = Optimization_utils.shift_charging_sessions(stationdata, shift_percentage=30)


Shifted load and cost

In [ ]:
shifted_load, shifted_cost = Optimization_utils.calculate_grid_load_and_cost(shifted_stationdata,
                                                                                 electricity_pricing)


Plot results

In [ ]:
Optimization_utils.plot_comparison(original_load, shifted_load)